In [1]:
import sys
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt

from sklearn.metrics import f1_score, roc_curve
from sklearn.cluster import KMeans

from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.initializers import Initializer
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.engine.topology import Layer
from keras import backend as K
from keras.models import load_model

sys.path.insert(0,'../DEC-keras')
from DEC import DEC

sys.path.insert(0,'../experiments/dissolving')
from dissolving_utils import load_dec

Using TensorFlow backend.
/Users/dwright/anaconda2/envs/aind/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/dwright/anaconda2/envs/aind/lib/python3.6/site-packages/matplotlib/__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
def get_cluster_to_label_mapping_safe(y, y_pred, n_classes, n_clusters):
  """Enusre at least one cluster assigned to each label.
  """
  one_hot_encoded = np_utils.to_categorical(y, n_classes)

  cluster_to_label_mapping = []
  n_assigned_list = []
  majority_class_fractions = []
  majority_class_pred = np.zeros(y.shape)
  for cluster in range(n_clusters):
    cluster_indices = np.where(y_pred == cluster)[0]
    n_assigned_examples = cluster_indices.shape[0]
    n_assigned_list.append(n_assigned_examples)
    cluster_labels = one_hot_encoded[cluster_indices]
    cluster_label_fractions = np.mean(cluster_labels, axis=0)
    majority_cluster_class = np.argmax(cluster_label_fractions)
    cluster_to_label_mapping.append(majority_cluster_class)
    majority_class_pred[cluster_indices] += majority_cluster_class
    majority_class_fractions.append(cluster_label_fractions[majority_cluster_class])
    print(cluster, n_assigned_examples, majority_cluster_class, cluster_label_fractions[majority_cluster_class])
  #print(cluster_to_label_mapping)

  print(np.unique(y), np.unique(cluster_to_label_mapping))
  try:
    # make sure there is at least 1 cluster representing each class
    assert np.all(np.unique(y) == np.unique(cluster_to_label_mapping))
  except AssertionError:
    # if there is no cluster for a class then we will assign a cluster to that
    # class
    
    # find which class it is
    # ASSUMPTION - this task is binary
    
    diff = list(set(np.unique(y)) - set(np.unique(cluster_to_label_mapping)))[0]
      # we choose the cluster that contains the most examples of the class with no cluster
      
    one_hot = np_utils.to_categorical(y_pred[np.where(y==diff)[0]], \
                                        len(cluster_to_label_mapping))
                                      
    cluster_to_label_mapping[np.argmax(np.sum(one_hot, axis=0))] = int(diff)
  print(cluster_to_label_mapping)
  return cluster_to_label_mapping, n_assigned_list, majority_class_fractions

In [3]:
def load_dec(x, ae_weights, dec_weights, n_clusters, batch_size, lr, momentum):
  dec = DEC(dims=[x.shape[-1], 500, 500, 2000, 10], n_clusters=n_clusters, batch_size=batch_size)
  ae_weights = ae_weights
  dec.initialize_model(optimizer=SGD(lr=lr, momentum=momentum),
                       ae_weights=ae_weights,
                       x=x, loss='kld')
  dec.load_weights(dec_weights)
  dec.model.summary()
  return dec

In [4]:
def percent_fpr(y, pred, fom):
  fpr, tpr, thresholds = roc_curve(y, pred)
  FoM = 1-tpr[np.where(fpr<=fom)[0][-1]] # MDR at 1% FPR
  threshold = thresholds[np.where(fpr<=fom)[0][-1]]
  return FoM, threshold, fpr, tpr

In [5]:
class MapInitializer(Initializer):
    
  def __init__(self, mapping, n_classes):
    self.mapping = mapping
    self.n_classes = n_classes

  def __call__(self, shape, dtype=None):
    return K.one_hot(self.mapping, self.n_classes)
    #return K.ones(shape=(100,10))

  def get_config(self):
    return {'mapping': self.mapping, 'n_classes': self.n_classes}

class MappingLayer(Layer):

  def __init__(self, mapping, output_dim, kernel_initializer, **kwargs):
  #def __init__(self, mapping, output_dim, **kwargs):
    self.output_dim = output_dim
    # mapping is a list where the index corresponds to a cluster and the value is the label.
    # e.g. say mapping[0] = 5, then a label of 5 has been assigned to cluster 0
    self.n_classes = np.unique(mapping).shape[0]      # get the number of classes
    self.mapping = K.variable(mapping, dtype='int32')
    self.kernel_initializer = kernel_initializer
    super(MappingLayer, self).__init__(**kwargs)

  def build(self, input_shape):
  
    self.kernel = self.add_weight(name='kernel', 
                                  shape=(input_shape[1], self.output_dim),
                                  initializer=self.kernel_initializer,
                                  trainable=False)
  
    super(MappingLayer, self).build(input_shape)  # Be sure to call this somewhere!

  def call(self, x):
    return K.softmax(K.dot(x, self.kernel))

  def compute_output_shape(self, input_shape):
    return (input_shape[0], self.output_dim)

In [6]:
class MultitaskDEC(DEC):
  def clustering(self, x, y=None, validation_data=None, tol=1e-3, update_interval=140, maxiter=2e4, save_dir='./results/dec', pretrained_weights=None):
    print('Update interval', update_interval)
    save_interval = x.shape[0] / self.batch_size * 5  # 5 epochs
    print('Save interval', save_interval)

    try:
      self.load_weights(pretrained_weights)
    except AttributeError:
      # initialize cluster centers using k-means
      print('Initializing cluster centers with k-means.')
      kmeans = KMeans(n_clusters=self.n_clusters, n_init=20)
      y_pred = kmeans.fit_predict(self.encoder.predict(x))
      y_pred_last = y_pred
      self.model.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])

    y_p = self.predict_clusters(x)
    self.n_classes = y.shape[1]
    
    # ensure at least one cluster assigned to each class
    cluster_to_label_mapping, n_assigned_list, majority_class_fractions = \
      get_cluster_to_label_mapping_safe(y[:,1], y_p, self.n_classes, self.n_clusters)
    
    # hack - ensure the cluster with the most real examples is assigned to the real class.
    #print(np.argmax((1-np.array(majority_class_fractions))*np.array(n_assigned_list)))
    #cluster_to_label_mapping[np.argmax((1-np.array(majority_class_fractions))*np.array(n_assigned_list))] = 1
    
    a = Input(shape=(x.shape[1],)) # input layer
    q_out = self.model(a)
    pred = MappingLayer(cluster_to_label_mapping, output_dim=self.n_classes, \
      name='mapping', kernel_initializer=MapInitializer(cluster_to_label_mapping, self.n_classes))(q_out)
    self.model = Model(inputs=a, outputs=[pred, q_out])

    optimizer = 'adam'
    self.model.compile(optimizer=optimizer, loss={'mapping': 'categorical_crossentropy', 'model_3': 'kld'}, \
                                      loss_weights={'mapping': 1, 'model_3': 1})

    loss = [np.inf, np.inf, np.inf]
    index = 0
    q = self.model.predict(x, verbose=0)[1]
    y_pred_last = q.argmax(1)
    best_val_loss = [np.inf, np.inf, np.inf]
    for ite in range(int(maxiter)):
      if ite % update_interval == 0:
        q = self.model.predict(x, verbose=0)[1]
        p = self.target_distribution(q)  # update the auxiliary target distribution p

        # evaluate the clustering performance
        y_pred = q.argmax(1)
        delta_label = np.sum(y_pred != y_pred_last).astype(np.float32) / y_pred.shape[0]
        y_pred_last = y_pred
        y_pred = self.model.predict(x)[0]
        if y is not None:
          loss = np.round(loss, 5)
          valid_p = self.target_distribution(self.model.predict(validation_data[0], verbose=0)[1])
          val_loss = np.round(self.model.test_on_batch(validation_data[0], [validation_data[1], valid_p]), 5)
          f, _, _, _ = percent_fpr(y[:,1], y_pred[:,1], 0.1)
          f = np.round(f, 5)
          f1 = np.round(f1_score(y[:,1], np.argmax(y_pred, axis=1)), 5)
          y_pred_valid = self.model.predict(validation_data[0])[0]
          f_valid, _, _, _ = percent_fpr(validation_data[1][:,1], y_pred_valid[:,1], 0.1)
          f_valid = np.round(f_valid, 5)
          f1_valid = np.round(f1_score(validation_data[1][:,1], np.argmax(y_pred_valid, axis=1)), 5)
          print('Iter', ite, ' :MDR at 10% FPR', f, ', F1=', f1, '; loss=', loss, \
                '; valid_loss=,', val_loss, '; valid MDR at 10% FPR=,', f_valid, ', valid F1=', f1_valid)
          if val_loss[1] < best_val_loss[1]: # only interested in classification improvements
            print('saving model: ', best_val_loss, ' -> ', val_loss)
            self.model.save_weights('best_val_loss.hf')
            best_val_loss = val_loss
      
        # train on batch
        if (index + 1) * self.batch_size > x.shape[0]:
          loss = self.model.train_on_batch(x=x[index * self.batch_size::],
                                           y=[y[index * self.batch_size::], \
                                              p[index * self.batch_size::]])
          index = 0
        else:
          loss = self.model.train_on_batch(x=x[index * self.batch_size:(index + 1) * self.batch_size],
                                           y=[y[index * self.batch_size:(index + 1) * self.batch_size], \
                                              p[index * self.batch_size:(index + 1) * self.batch_size]])
          index += 1

        # save intermediate model
        if ite % save_interval == 0:
        # save IDEC model checkpoints
          print('saving model to:', save_dir + '/DEC_model_' + str(ite) + '.h5')
          self.model.save_weights(save_dir + '/DEC_model_' + str(ite) + '.h5')

        ite += 1

    # save the trained model
    print('saving model to:', save_dir + '/DEC_model_final.h5')
    self.model.save_weights(save_dir + '/DEC_model_final.h5')

    return y_pred

  def predict_clusters(self, x):  # predict cluster labels using the output of clustering layer
    q = self.model.predict(x, verbose=0)
    try:
      return q.argmax(1)
    except AttributeError:
      return q[1].argmax(1)

### Load in some volunteer labelled training data

In [7]:
for i in range(1,31):
  data = sio.loadmat('../data/snhunters/3pi_20x20_supernova_hunters_batch_%d_signPreserveNorm_detect_misaligned.mat'%(i))
  try:
    x_train = np.concatenate((x_train, np.nan_to_num(np.reshape(data['X'], \
      (data['X'].shape[0], 400), order='F'))))
    y_train = np.concatenate((y_train, np.squeeze(data['y'])))
  except NameError:
    x_train = np.nan_to_num(np.reshape(data['X'], (data['X'].shape[0], 400), \
      order='F'))
    y_train = np.squeeze(data['y'])

Only use the first classifications of subject.

In [8]:
u, indices = np.unique(x_train, return_index=True, axis=0)
x_train = x_train[indices]
y_train = y_train[indices]

# divide into training and validation sets
m = x_train.shape[0]
split = int(.75*m)

x_valid = x_train[split:]
y_valid = y_train[split:]

x_train = x_train[:split]
y_train = y_train[:split]

### Set up the multitask DEC initiaising it with the previously trainined weights.

In [9]:
ae_weights = '../DEC-keras/results/snh/ae_weights_snh.h5' # previously trained Auto-encoder weights
dec_weights = '../DEC-keras/results/snh/10/DEC_model_final.h5' # previously trained DEC weights

In [10]:
dec = MultitaskDEC(dims=[x_train.shape[-1], 500, 500, 2000, 10], \
                   n_clusters=10, batch_size=256)
dec.initialize_model(optimizer=SGD(lr=0.01, momentum=0.9),
                     ae_weights=ae_weights,
                     x=x_train)
dec.model.load_weights(dec_weights)

### Retrain the DEC model using the volunteer labels

Only running for 1000 iterations here

In [11]:
dec.clustering(x_train, np_utils.to_categorical(y_train), \
              (x_valid, np_utils.to_categorical(y_valid)), \
               pretrained_weights=dec_weights, maxiter=1000)

Update interval 140
Save interval 454.21875
0 140 0 0.75
1 965 0 0.6093264248704663
2 1395 0 0.589247311827957
3 6120 0 0.6168300653594772
4 2072 0 0.6013513513513513
5 2610 0 0.542911877394636
6 4231 0 0.6343653982510045
7 4571 0 0.6226208707066287
8 4 1 0.75
9 1148 0 0.578397212543554
[0. 1.] [0 1]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0]


/Users/dwright/anaconda2/envs/aind/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Iter 0  :MDR at 10% FPR 0.90217 , F1= 0.0 ; loss= [inf inf inf] ; valid_loss=, [0.77632 0.67048 0.10584] ; valid MDR at 10% FPR=, 0.91792 , valid F1= 0.0
saving model:  [inf, inf, inf]  ->  [0.77632 0.67048 0.10584]
saving model to: ./results/dec/DEC_model_0.h5
Iter 140  :MDR at 10% FPR 0.90086 , F1= 0.0 ; loss= [0.77405 0.6738  0.10025] ; valid_loss=, [0.87027 0.66555 0.20472] ; valid MDR at 10% FPR=, 0.92652 , valid F1= 0.0
saving model:  [0.77632 0.67048 0.10584]  ->  [0.87027 0.66555 0.20472]
Iter 280  :MDR at 10% FPR 0.89756 , F1= 0.0 ; loss= [0.81072 0.60038 0.21034] ; valid_loss=, [0.8455  0.66807 0.17743] ; valid MDR at 10% FPR=, 0.92222 , valid F1= 0.0
Iter 420  :MDR at 10% FPR 0.90821 , F1= 0.0 ; loss= [0.836   0.66175 0.17425] ; valid_loss=, [0.8486  0.66799 0.1806 ] ; valid MDR at 10% FPR=, 0.92079 , valid F1= 0.0
Iter 560  :MDR at 10% FPR 0.90075 , F1= 0.0 ; loss= [0.86913 0.69609 0.17304] ; valid_loss=, [0.83977 0.66841 0.17136] ; valid MDR at 10% FPR=, 0.92043 , valid F1

array([[0.72151625, 0.27848372],
       [0.72716945, 0.2728305 ],
       [0.7108438 , 0.28915617],
       ...,
       [0.7234734 , 0.2765265 ],
       [0.722374  , 0.277626  ],
       [0.72217363, 0.27782634]], dtype=float32)